In [1]:
! free -m

              total        used        free      shared  buff/cache   available
Mem:          13029         389         934           0       11705       12391
Swap:             0           0           0


In [2]:
!ls

datalab


I used  TensorFlow [word2vec tutoria](https://www.tensorflow.org/tutorials/word2vec)l to code this. Have a good read before trying to understand my code. 

**Skip-gram word2vec for Sanskrit**

I'll lead you through using TensorFlow to implement the word2vec algorithm for Sanskrit using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will be very useful for machine translation . Till now translators have not been developed for most ancient Indian language Sanskrit. Here is noble attempt which will serve as baby steps for future development of machine translation in Sanskrit language.

The dataset which I have used has been compilled using [sanskrit_text_jnu](https://github.com/cltk/sanskrit_text_jnu.git)  [ sanskrit_parallel_sacred_texts](https://github.com/cltk/sanskrit_parallel_sacred_texts.git)  [sanskrit_text_sacred_texts](https://github.com/cltk/sanskrit_text_sacred_texts.git)  [sanskrit_text_sanskrit_documents](https://github.com/cltk/sanskrit_text_sanskrit_documents.git)  [sanskrit_text_gitasupersite](https://github.com/cltk/sanskrit_text_gitasupersite.git)  [ sanskrit_parallel_gitasupersite](https://github.com/cltk/sanskrit_parallel_gitasupersite.git)

In [0]:
import numpy as np 
import collections
import math
import random
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf


In [4]:
! wget https://www.dropbox.com/s/u2z4rhv4juzvof3/w2v.txt

--2018-06-22 06:37:16--  https://www.dropbox.com/s/u2z4rhv4juzvof3/w2v.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb77b922ade778e19091dd52f7f.dl.dropboxusercontent.com/cd/0/get/AJcIZkxrd1uK6FnFtgyI0FCxJuz48DfE5OvZJVsSijd66KoV_tu7FATiAk2r70UchbudPsNm8EhmtS6Zz9FjDd-YnAFn_u9wuxYXOh1cT-Gxs-rOlVydxonlZeBjGdjeQA4p1YbhYSbIkktp-h8thC1gCRDQK7mqNjREy30Lo5macyKDA4jCkjLLRHzxzeD8kTo/file [following]
--2018-06-22 06:37:17--  https://ucb77b922ade778e19091dd52f7f.dl.dropboxusercontent.com/cd/0/get/AJcIZkxrd1uK6FnFtgyI0FCxJuz48DfE5OvZJVsSijd66KoV_tu7FATiAk2r70UchbudPsNm8EhmtS6Zz9FjDd-YnAFn_u9wuxYXOh1cT-Gxs-rOlVydxonlZeBjGdjeQA4p1YbhYSbIkktp-h8thC1gCRDQK7mqNjREy30Lo5macyKDA4jCkjLLRHzxzeD8kTo/file
Resolving ucb77b922ade778e19091dd52f7f.dl.dropboxusercontent.com (ucb77b922ade778e19091dd52f7f.dl.dropboxuserconten

In [0]:
h=open('w2v.txt', encoding="utf8")
v=h.read()
h.close()
vocabulary=v.split()

In [7]:
vocabulary

['गुरूभक्तः',
 'आरूणिः',
 'प्राचीनकालस्य',
 'कथा',
 'अस्ति',
 'एकस्मिन्',
 'आश्रमे',
 'ॠषिः',
 'धौम्यः',
 'शिष्यैः',
 'सह',
 'वसति',
 'स्म',
 'ते',
 'शिष्याः',
 'आश्रमं',
 'निकषा',
 'कृषिक्षेत्रेषु',
 'कृषिकार्यमपि',
 'कुर्वन्ति',
 'स्म',
 'तेषु',
 'एकः',
 'अतीव',
 'सरलः',
 'बालः',
 'आसीत्',
 'तस्य',
 'नाम',
 'आरूणि',
 'आसीत्',
 'वर्षाकाले',
 'एकस्मिन्',
 'दिवसे',
 'प्रचण्डा',
 'वृष्टिरभवत्',
 'पूर्वं',
 'धान्यं',
 'रोपितमासीत्',
 'अतः',
 'तत्र',
 'जलं',
 'रक्षणीयमासीत्',
 'गुरू',
 'धौम्यः',
 'आरूणिम्',
 'आहूय',
 'अवदत्आरूणे',
 'धान्यक्षेत्रं',
 'गच्छ',
 'तत्र',
 'जलस्य',
 'रक्षणं',
 'कुरू',
 'त्वरितमेव',
 'आरूणिः',
 'धान्यक्षेत्रं',
 'प्रति',
 'अगच्छत्',
 'एकस्मिन्',
 'स्थले',
 'क्षेत्रबन्धः',
 'भग्नः',
 'आसीत्',
 'तेन',
 'कारणेन',
 'क्षेत्रस्य',
 'जलं',
 'बहिः',
 'निर्गच्छति',
 'स्म',
 'जलस्य',
 'रक्षणार्थं',
 'क्षेत्रस्य',
 'बन्धः',
 'अवश्यं',
 'रक्षणीयः',
 'इत्यारूणिः',
 'अचिन्तयत्',
 'भग्ने',
 'क्षेत्रबन्धे',
 'आरूणिः',
 'मृत्तिकां',
 'स्थापितवान्',
 'परन्तु',
 'त्वरितमेव',
 'उग्र

In [9]:
len(vocabulary)

2368513

In [14]:
len(set(vocabulary))

509129

In [0]:
vocabulary_size=500000

In [13]:
def build_dataset(words, n_words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                            vocabulary_size)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 9130], ('च', 65200), ('न', 28380), ('स', 19500), ('तु', 15647)]
Sample data [51945, 6673, 51946, 568, 207, 4915, 2681, 51947, 10027, 14988] ['गुरूभक्तः', 'आरूणिः', 'प्राचीनकालस्य', 'कथा', 'अस्ति', 'एकस्मिन्', 'आश्रमे', 'ॠषिः', 'धौम्यः', 'शिष्यैः']


In [18]:
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window 
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

51947 ॠषिः -> 2681 आश्रमे
51947 ॠषिः -> 10027 धौम्यः
10027 धौम्यः -> 51947 ॠषिः
10027 धौम्यः -> 14988 शिष्यैः
14988 शिष्यैः -> 10027 धौम्यः
14988 शिष्यैः -> 32 सह
32 सह -> 1494 वसति
32 सह -> 14988 शिष्यैः


 The window we createcaptures the semantics in the sentence that which word is occuring before after this. We minimixe the negative log probability of the occurence of context word with reference to the target word.

In [0]:
batch_size = 500
embedding_size = 300  # This is length of the word vecor.Every word will be representes in these many dimensions.
skip_window = 5       # How many words to consider left and right.
num_skips = 5         # How many times to reuse an input to generate a label.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.


In [0]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()

In [45]:
num_steps = 200001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
                                 batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
          print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  377.03582763671875
Nearest to तान: सिद्धदेवर्षिदयितं,
Nearest to तान: सिद्धदेवर्षिदयितं, रूढानामपि,
Nearest to तान: सिद्धदेवर्षिदयितं, रूढानामपि, माधुर्या,
Nearest to तान: सिद्धदेवर्षिदयितं, रूढानामपि, माधुर्या, नानानगरमालिनीम्,
Nearest to तान: सिद्धदेवर्षिदयितं, रूढानामपि, माधुर्या, नानानगरमालिनीम्, परशाखा,
Nearest to तान: सिद्धदेवर्षिदयितं, रूढानामपि, माधुर्या, नानानगरमालिनीम्, परशाखा, ककुत्,
Nearest to तान: सिद्धदेवर्षिदयितं, रूढानामपि, माधुर्या, नानानगरमालिनीम्, परशाखा, ककुत्, सवृक्षक्षुप,
Nearest to तान: सिद्धदेवर्षिदयितं, रूढानामपि, माधुर्या, नानानगरमालिनीम्, परशाखा, ककुत्, सवृक्षक्षुप, विविधैर्गन्धैर्धूपैश्चागरुगन्धिभि:,
Nearest to नाम: शय्यासनाटनस्थानस्नानक्रीडाशनादिषु,
Nearest to नाम: शय्यासनाटनस्थानस्नानक्रीडाशनादिषु, धनुश्च्युतानां,
Nearest to नाम: शय्यासनाटनस्थानस्नानक्रीडाशनादिषु, धनुश्च्युतानां, यदर्धमायुषस्तस्य,
Nearest to नाम: शय्यासनाटनस्थानस्नानक्रीडाशनादिषु, धनुश्च्युतानां, यदर्धमायुषस्तस्य, ऽपान,
Nearest to नाम: शय्यासनाटनस्थान

Nearest to चिद: ये,
Nearest to चिद: ये, शिरो,
Nearest to चिद: ये, शिरो, अहं,
Nearest to चिद: ये, शिरो, अहं, सूर्यो,
Nearest to चिद: ये, शिरो, अहं, सूर्यो, जायते,
Nearest to चिद: ये, शिरो, अहं, सूर्यो, जायते, चित,
Nearest to चिद: ये, शिरो, अहं, सूर्यो, जायते, चित, विना,
Nearest to चिद: ये, शिरो, अहं, सूर्यो, जायते, चित, विना, पाहि,
Nearest to सह: चैव,
Nearest to सह: चैव, जगत्,
Nearest to सह: चैव, जगत्, शक्यते,
Nearest to सह: चैव, जगत्, शक्यते, नराः,
Nearest to सह: चैव, जगत्, शक्यते, नराः, कृष्णं,
Nearest to सह: चैव, जगत्, शक्यते, नराः, कृष्णं, तल्लक्षणं,
Nearest to सह: चैव, जगत्, शक्यते, नराः, कृष्णं, तल्लक्षणं, एष,
Nearest to सह: चैव, जगत्, शक्यते, नराः, कृष्णं, तल्लक्षणं, एष, तथापि,
Nearest to पुरा: संशयः,
Nearest to पुरा: संशयः, नाम्ना,
Nearest to पुरा: संशयः, नाम्ना, पुत्र,
Nearest to पुरा: संशयः, नाम्ना, पुत्र, सर्वान्,
Nearest to पुरा: संशयः, नाम्ना, पुत्र, सर्वान्, निहतं,
Nearest to पुरा: संशयः, नाम्ना, पुत्र, सर्वान्, निहतं, नित्यं,
Nearest to पुरा: संशयः, नाम्ना, पुत्र, सर्वान्

Average loss at step  28000 :  85.60675284481049
Average loss at step  30000 :  84.48314593887329
Nearest to तान: श्रीगारुडे,
Nearest to तान: श्रीगारुडे, यान्ति,
Nearest to तान: श्रीगारुडे, यान्ति, सॊमकान,
Nearest to तान: श्रीगारुडे, यान्ति, सॊमकान, परियम,
Nearest to तान: श्रीगारुडे, यान्ति, सॊमकान, परियम, वासुदेवः,
Nearest to तान: श्रीगारुडे, यान्ति, सॊमकान, परियम, वासुदेवः, महारथाः,
Nearest to तान: श्रीगारुडे, यान्ति, सॊमकान, परियम, वासुदेवः, महारथाः, दृष्टः,
Nearest to तान: श्रीगारुडे, यान्ति, सॊमकान, परियम, वासुदेवः, महारथाः, दृष्टः, नर,
Nearest to नाम: चेत,
Nearest to नाम: चेत, पश्यध्वं,
Nearest to नाम: चेत, पश्यध्वं, कौन्तेयं,
Nearest to नाम: चेत, पश्यध्वं, कौन्तेयं, मान,
Nearest to नाम: चेत, पश्यध्वं, कौन्तेयं, मान, कीर्तितं,
Nearest to नाम: चेत, पश्यध्वं, कौन्तेयं, मान, कीर्तितं, रुद्रः,
Nearest to नाम: चेत, पश्यध्वं, कौन्तेयं, मान, कीर्तितं, रुद्रः, भय,
Nearest to नाम: चेत, पश्यध्वं, कौन्तेयं, मान, कीर्तितं, रुद्रः, भय, इन्द्रः,
Nearest to तव: च,
Nearest to तव: च, स,
Nearest t

Average loss at step  44000 :  61.92770583319664
Average loss at step  46000 :  53.059709278583526
Average loss at step  48000 :  54.7196349132061
Average loss at step  50000 :  61.72207000875473
Nearest to तान: सॊमकान,
Nearest to तान: सॊमकान, श्रीगारुडे,
Nearest to तान: सॊमकान, श्रीगारुडे, यान्ति,
Nearest to तान: सॊमकान, श्रीगारुडे, यान्ति, तरीण्य,
Nearest to तान: सॊमकान, श्रीगारुडे, यान्ति, तरीण्य, माधुर्या,
Nearest to तान: सॊमकान, श्रीगारुडे, यान्ति, तरीण्य, माधुर्या, वह्नौ,
Nearest to तान: सॊमकान, श्रीगारुडे, यान्ति, तरीण्य, माधुर्या, वह्नौ, वासुदेवः,
Nearest to तान: सॊमकान, श्रीगारुडे, यान्ति, तरीण्य, माधुर्या, वह्नौ, वासुदेवः, दरौणेर,
Nearest to नाम: पश्यध्वं,
Nearest to नाम: पश्यध्वं, कीर्तितं,
Nearest to नाम: पश्यध्वं, कीर्तितं, मान,
Nearest to नाम: पश्यध्वं, कीर्तितं, मान, चेत,
Nearest to नाम: पश्यध्वं, कीर्तितं, मान, चेत, ऽपान,
Nearest to नाम: पश्यध्वं, कीर्तितं, मान, चेत, ऽपान, कौन्तेयं,
Nearest to नाम: पश्यध्वं, कीर्तितं, मान, चेत, ऽपान, कौन्तेयं, भय,
Nearest to नाम: पश्यध्

Average loss at step  66000 :  38.50478829479218
Average loss at step  68000 :  34.72784498548508
Average loss at step  70000 :  30.69111904168129
Nearest to तान: च,
Nearest to तान: च, सॊमकान,
Nearest to तान: च, सॊमकान, UNK,
Nearest to तान: च, सॊमकान, UNK, तु,
Nearest to तान: च, सॊमकान, UNK, तु, यान्ति,
Nearest to तान: च, सॊमकान, UNK, तु, यान्ति, तरीण्य,
Nearest to तान: च, सॊमकान, UNK, तु, यान्ति, तरीण्य, वाक्यं,
Nearest to तान: च, सॊमकान, UNK, तु, यान्ति, तरीण्य, वाक्यं, माधुर्या,
Nearest to नाम: ऽपान,
Nearest to नाम: ऽपान, कीर्तितं,
Nearest to नाम: ऽपान, कीर्तितं, पश्यध्वं,
Nearest to नाम: ऽपान, कीर्तितं, पश्यध्वं, चेत,
Nearest to नाम: ऽपान, कीर्तितं, पश्यध्वं, चेत, तद,
Nearest to नाम: ऽपान, कीर्तितं, पश्यध्वं, चेत, तद, मान,
Nearest to नाम: ऽपान, कीर्तितं, पश्यध्वं, चेत, तद, मान, कौन्तेयं,
Nearest to नाम: ऽपान, कीर्तितं, पश्यध्वं, चेत, तद, मान, कौन्तेयं, इन्द्रः,
Nearest to तव: तु,
Nearest to तव: तु, स,
Nearest to तव: तु, स, ते,
Nearest to तव: तु, स, ते, च,
Nearest to तव: तु, स, ते, 

...

Average loss at step  108000 :  18.574952717065813
Average loss at step  110000 :  17.210414999485018
Nearest to तान: तु,
Nearest to तान: तु, मे,
Nearest to तान: तु, मे, वाक्यं,
Nearest to तान: तु, मे, वाक्यं, ते,
Nearest to तान: तु, मे, वाक्यं, ते, UNK,
Nearest to तान: तु, मे, वाक्यं, ते, UNK, च,
Nearest to तान: तु, मे, वाक्यं, ते, UNK, च, यान्ति,
Nearest to तान: तु, मे, वाक्यं, ते, UNK, च, यान्ति, ततः,
Nearest to नाम: UNK,
Nearest to नाम: UNK, न,
Nearest to नाम: UNK, न, ऽपान,
Nearest to नाम: UNK, न, ऽपान, तद,
Nearest to नाम: UNK, न, ऽपान, तद, च,
Nearest to नाम: UNK, न, ऽपान, तद, च, हि,
Nearest to नाम: UNK, न, ऽपान, तद, च, हि, कीर्तितं,
Nearest to नाम: UNK, न, ऽपान, तद, च, हि, कीर्तितं, पश्यध्वं,
Nearest to तव: तु,
Nearest to तव: तु, UNK,
Nearest to तव: तु, UNK, स,
Nearest to तव: तु, UNK, स, न,
Nearest to तव: तु, UNK, स, न, च,
Nearest to तव: तु, UNK, स, न, च, ते,
Nearest to तव: तु, UNK, स, न, च, ते, तथा,
Nearest to तव: तु, UNK, स, न, च, ते, तथा, यथा,
Nearest to यद: न,
Nearest to यद: न

Average loss at step  128000 :  14.391574661016465
Average loss at step  130000 :  13.857458235025407
Nearest to तान: तु,
Nearest to तान: तु, वाक्यं,
Nearest to तान: तु, वाक्यं, मे,
Nearest to तान: तु, वाक्यं, मे, ते,
Nearest to तान: तु, वाक्यं, मे, ते, ततः,
Nearest to तान: तु, वाक्यं, मे, ते, ततः, यान्ति,
Nearest to तान: तु, वाक्यं, मे, ते, ततः, यान्ति, अब्रवीत,
Nearest to तान: तु, वाक्यं, मे, ते, ततः, यान्ति, अब्रवीत, इति,
Nearest to नाम: UNK,
Nearest to नाम: UNK, न,
Nearest to नाम: UNK, न, ऽपान,
Nearest to नाम: UNK, न, ऽपान, च,
Nearest to नाम: UNK, न, ऽपान, च, यदर्धमायुषस्तस्य,
Nearest to नाम: UNK, न, ऽपान, च, यदर्धमायुषस्तस्य, कीर्तितं,
Nearest to नाम: UNK, न, ऽपान, च, यदर्धमायुषस्तस्य, कीर्तितं, तद,
Nearest to नाम: UNK, न, ऽपान, च, यदर्धमायुषस्तस्य, कीर्तितं, तद, पश्यध्वं,
Nearest to तव: तु,
Nearest to तव: तु, UNK,
Nearest to तव: तु, UNK, स,
Nearest to तव: तु, UNK, स, च,
Nearest to तव: तु, UNK, स, च, न,
Nearest to तव: तु, UNK, स, च, न, ते,
Nearest to तव: तु, UNK, स, च, न, ते, तथा,

Average loss at step  148000 :  11.433857036173343
Average loss at step  150000 :  11.841386692762375
Nearest to तान: वाक्यं,
Nearest to तान: वाक्यं, तु,
Nearest to तान: वाक्यं, तु, मे,
Nearest to तान: वाक्यं, तु, मे, यान्ति,
Nearest to तान: वाक्यं, तु, मे, यान्ति, UNK,
Nearest to तान: वाक्यं, तु, मे, यान्ति, UNK, अब्रवीत,
Nearest to तान: वाक्यं, तु, मे, यान्ति, UNK, अब्रवीत, सॊमकान,
Nearest to तान: वाक्यं, तु, मे, यान्ति, UNK, अब्रवीत, सॊमकान, माधुर्या,
Nearest to नाम: UNK,
Nearest to नाम: UNK, ऽपान,
Nearest to नाम: UNK, ऽपान, यदर्धमायुषस्तस्य,
Nearest to नाम: UNK, ऽपान, यदर्धमायुषस्तस्य, कीर्तितं,
Nearest to नाम: UNK, ऽपान, यदर्धमायुषस्तस्य, कीर्तितं, न,
Nearest to नाम: UNK, ऽपान, यदर्धमायुषस्तस्य, कीर्तितं, न, धनुश्च्युतानां,
Nearest to नाम: UNK, ऽपान, यदर्धमायुषस्तस्य, कीर्तितं, न, धनुश्च्युतानां, चेत,
Nearest to नाम: UNK, ऽपान, यदर्धमायुषस्तस्य, कीर्तितं, न, धनुश्च्युतानां, चेत, गृष्टिं,
Nearest to तव: तु,
Nearest to तव: तु, UNK,
Nearest to तव: तु, UNK, स,
Nearest to तव: तु, UNK, 

Average loss at step  166000 :  9.766430151820183
Average loss at step  168000 :  10.494085725426674
Average loss at step  170000 :  9.6710381680727
Nearest to तान: वाक्यं,
Nearest to तान: वाक्यं, तु,
Nearest to तान: वाक्यं, तु, माधुर्या,
Nearest to तान: वाक्यं, तु, माधुर्या, अब्रवीत,
Nearest to तान: वाक्यं, तु, माधुर्या, अब्रवीत, सॊमकान,
Nearest to तान: वाक्यं, तु, माधुर्या, अब्रवीत, सॊमकान, अवध्येन,
Nearest to तान: वाक्यं, तु, माधुर्या, अब्रवीत, सॊमकान, अवध्येन, मे,
Nearest to तान: वाक्यं, तु, माधुर्या, अब्रवीत, सॊमकान, अवध्येन, मे, यान्ति,
Nearest to नाम: ऽपान,
Nearest to नाम: ऽपान, न,
Nearest to नाम: ऽपान, न, यदर्धमायुषस्तस्य,
Nearest to नाम: ऽपान, न, यदर्धमायुषस्तस्य, कीर्तितं,
Nearest to नाम: ऽपान, न, यदर्धमायुषस्तस्य, कीर्तितं, धनुश्च्युतानां,
Nearest to नाम: ऽपान, न, यदर्धमायुषस्तस्य, कीर्तितं, धनुश्च्युतानां, गृष्टिं,
Nearest to नाम: ऽपान, न, यदर्धमायुषस्तस्य, कीर्तितं, धनुश्च्युतानां, गृष्टिं, कौन्तेयं,
Nearest to नाम: ऽपान, न, यदर्धमायुषस्तस्य, कीर्तितं, धनुश्च्युतानां, गृष्

Average loss at step  186000 :  8.683528204798698
Average loss at step  188000 :  8.128262855648995
Average loss at step  190000 :  8.451172679781914
Nearest to तान: वाक्यं,
Nearest to तान: वाक्यं, तु,
Nearest to तान: वाक्यं, तु, माधुर्या,
Nearest to तान: वाक्यं, तु, माधुर्या, अवध्येन,
Nearest to तान: वाक्यं, तु, माधुर्या, अवध्येन, सॊमकान,
Nearest to तान: वाक्यं, तु, माधुर्या, अवध्येन, सॊमकान, परशाखा,
Nearest to तान: वाक्यं, तु, माधुर्या, अवध्येन, सॊमकान, परशाखा, अब्रवीत,
Nearest to तान: वाक्यं, तु, माधुर्या, अवध्येन, सॊमकान, परशाखा, अब्रवीत, सवृक्षक्षुप,
Nearest to नाम: ऽपान,
Nearest to नाम: ऽपान, कीर्तितं,
Nearest to नाम: ऽपान, कीर्तितं, यदर्धमायुषस्तस्य,
Nearest to नाम: ऽपान, कीर्तितं, यदर्धमायुषस्तस्य, UNK,
Nearest to नाम: ऽपान, कीर्तितं, यदर्धमायुषस्तस्य, UNK, धनुश्च्युतानां,
Nearest to नाम: ऽपान, कीर्तितं, यदर्धमायुषस्तस्य, UNK, धनुश्च्युतानां, सांपराय,
Nearest to नाम: ऽपान, कीर्तितं, यदर्धमायुषस्तस्य, UNK, धनुश्च्युतानां, सांपराय, गृष्टिं,
Nearest to नाम: ऽपान, कीर्तितं, यदर्धमा

In [0]:
fe=np.array(final_embeddings)

In [47]:
fe.shape

(500000, 300)

In [48]:
fe

array([[-0.01635464,  0.01859653, -0.06892549, ..., -0.02755348,
        -0.0526773 , -0.04340959],
       [ 0.00516842, -0.02553275, -0.14012717, ..., -0.01757955,
        -0.03935834, -0.10511653],
       [-0.03094754, -0.0048249 , -0.09965251, ..., -0.08898933,
        -0.1499638 , -0.03822214],
       ...,
       [ 0.00755448,  0.04274349,  0.02144615, ..., -0.01611131,
        -0.06758233,  0.00056627],
       [-0.05216298, -0.08306621,  0.05700618, ..., -0.07402141,
        -0.0298771 ,  0.00103917],
       [ 0.08163057,  0.04463291, -0.08604634, ...,  0.03796234,
        -0.08789171, -0.06965437]], dtype=float32)

In [50]:
print(vocabulary[490000:500000])

['मल्लयुद्धे', 'तौ', 'घातयिष्यामि', 'दुर्मदौ', 'धनुर्महमहायाग', 'व्याजेनानीय', 'तौ', 'व्रजात्', 'तथा', 'तथा', 'करिष्यामि', 'यास्यतः', 'सङ्क्षयं', 'यथा', 'व्यास', 'उवाच', 'इत्यालोच्य', 'स', 'दुष्टात्मा', 'कंसो', 'रामजनार्दनौ', 'हन्तुं', 'कृतमतिर्वीरमक्रूरं', 'वाक्यमब्रवीत्', 'कंस', 'उवाच', 'भो', 'भो', 'दानपते', 'वाक्यं', 'क्रियतां', 'प्रीतये', 'मम', 'इतः', 'स्यन्दनमारुह्य', 'गम्यतां', 'नन्दगोकुलम्', 'वसुदेवसुतौ', 'तत्र', 'विष्णोरंशसमुद्भवौ', 'नाशाय', 'किल', 'सम्भूतौ', 'मम', 'दुष्टौ', 'प्रवर्धतः', 'धनुर्महमहायागश्चतुर्दश्यां', 'भविष्यति', 'आनेयौ', 'भवता', 'तौ', 'तु', 'मल्लयुद्धाय', 'तत्र', 'वै', 'चाणूरमुष्टिकौ', 'मल्लौ', 'नियुद्धकुशलौ', 'मम', 'ताभ्यां', 'सहानयोर्युद्धं', 'सर्वलोको', 'ऽत्र', 'पश्यतु', 'नागः', 'कुवलयापीडो', 'महामात्रप्रचोदितः', 'स', 'तौ', 'निहंस्यते', 'पापौ', 'वसुदेवात्मजौ', 'शिशू', 'तौ', 'हत्वा', 'वसुदेवं', 'च', 'नन्दगोपं', 'च', 'दुर्मतिम्', 'हनिष्ये', 'पितरं', 'चैव', 'उग्रसेनं', 'च', 'दुर्मतिम्', 'ततः', 'समस्तगोपानां', 'गोधनान्यखिलान्यहम्', 'वित्तं', 'चापहरिष्यामि', 'दुष